In [3]:
from XMTR import MTR
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [6]:
def calc_al_error(instance, perc):
  # the error should be non zero
  return (abs(instance)+0.1)*perc 


RF2_df = pd.read_csv('RF2.csv')
RF2_df = RF2_df.dropna(axis=0)

RF2_cols = RF2_df.columns
RF2_df = RF2_df[RF2_cols[1:]].reset_index(drop=True)
RF2_df = RF2_df.iloc[:500]

RF2_cols = RF2_df.columns
f_n = RF2_cols[:200]
t_n = RF2_cols[576:]
#f_n = f_n[:15]
#t_n = t_n[:5]

X = RF2_df[f_n]
y = RF2_df[t_n]

# convert to numpy
X = X.to_numpy()
y = y.to_numpy()

scaler = MinMaxScaler(feature_range=(1,10))
y = scaler.fit_transform(y)


# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)
allowed_error = 0.1
print(len(X_test))
max_ds = [5, 10]
min_s_ls = [5, 10]
m_fs = ['sqrt', 0.75, None]
ests = [100, 500, 1000]
total = (len(max_ds)*len(min_s_ls)*len(m_fs)*len(ests))
counter = 1
min_len = (len(f_n)+1) # just a ref, should be > than len(feature_names)
for max_d in max_ds:
  for min_s_l in min_s_ls:
    for m_f in m_fs:
      for est in ests:
        # create/train model
        RFmodel = RandomForestRegressor(n_estimators=est, max_depth=max_d, min_samples_leaf=min_s_l, max_features=m_f, n_jobs=-1, random_state=42)
        RFmodel.fit(X_train, y_train)
        MTR_obj = MTR(model=RFmodel, X_train = X_train, X_test=X_test, y_train=y_train, y_test=y_test, feature_names=f_n, target_names=t_n)
        avgRuleLengths = 0
        avgEstimators = []
        # get results
        for i in range(len(X_test)):
          error = calc_al_error(y_test[i], allowed_error)
          rule = MTR_obj.explain(X_test[i], error) # explain instance
          feature_limits = MTR_obj.getFeatureLimits()
          avgRuleLengths += len(feature_limits.keys())
          estimators = MTR_obj.getAmountOfReduction() # get estimators
          avgEstimators.append(estimators[0])
        finalLen = avgRuleLengths/len(X_test)
        if finalLen < min_len:
          min_len = finalLen
          best_par = {"max_d=":max_d, "min_s_l=":min_s_l, "m_f=": m_f, "est=": est, "rule length=": finalLen}
        print(counter, '/', total, "| max_depth=",max_d," | ", "min_sample_leaf=",min_s_l," | ", "max_f=", m_f," | ", "est=", est," | ", "rule length=", finalLen," | ", "estimators=", round(np.array(avgEstimators).mean(),3),"/",est)
        counter += 1

50
1 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 100  |  rule length= 86.94  |  estimators= 100.0 / 100
2 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 500  |  rule length= 130.16  |  estimators= 500.0 / 500
3 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 1000  |  rule length= 146.98  |  estimators= 1000.0 / 1000
4 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 100  |  rule length= 57.42  |  estimators= 100.0 / 100
5 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 500  |  rule length= 79.02  |  estimators= 500.0 / 500
6 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 1000  |  rule length= 88.46  |  estimators= 1000.0 / 1000
7 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= None  |  est= 100  |  rule length= 53.82  |  estimators= 100.0 / 100
8 / 36 | max_depth= 5  |  min_sample_leaf= 5  |  max_f= None  |  est= 500  |  rule length= 73.26  |  estimators= 5

In [7]:
best_par

{'max_d=': 5, 'min_s_l=': 10, 'm_f=': None, 'est=': 100, 'rule length=': 41.78}